### Metadata process of single file

In [8]:
#Env 1 Lat_Long files Source, Furuno GP170 GPS
import pandas as pd
from datetime import datetime

# Define the path to your raw file
file_path = 'C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/Lat_Long/GP170-GPGGA-RAW_20220511-000000.Raw'

# Initialize a list to hold the parsed data
data_list = []

# Open and read the raw file
with open(file_path, 'r') as file:
    for line in file:
        parts = line.strip().split(',')
        # Check if the line has the minimum expected number of parts to parse
        if len(parts) >= 7:
            # Extract datetime information
            date_str = parts[0]
            time_str = parts[1].split('.')[0]  # Exclude milliseconds for simplicity
            datetime_str = f"{date_str} {time_str}"
            datetime_obj = datetime.strptime(datetime_str, '%m/%d/%Y %H:%M:%S')
            
            # Extract latitude and longitude
            latitude = parts[4] + parts[5]  # e.g., "5350.7244N"
            longitude = parts[6] + parts[7]  # e.g., "16634.7584W"
            
            # Append the extracted data to the list
            data_list.append({
                'DateTime': datetime_obj,
                'Latitude': latitude,
                'Longitude': longitude
            })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data_list)

# Display the first few rows of the DataFrame
print(df.head())

# Optionally, save the DataFrame to a CSV file
output_csv_path = 'C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/Lat_Long/processed_Lat_long_20220511.csv'
df.to_csv(output_csv_path, index=False)


             DateTime    Latitude    Longitude
0 2022-05-11 00:00:00  5421.8252N  16616.0461W
1 2022-05-11 00:00:01  5421.8286N  16616.0436W
2 2022-05-11 00:00:02  5421.8318N  16616.0412W
3 2022-05-11 00:00:03  5421.8351N  16616.0385W
4 2022-05-11 00:00:04  5421.8383N  16616.0356W


In [27]:
#Env 2 file TSG files from Underway thermosalinograph (TSG). SBE45
#one csv file
import pandas as pd

# Initialize a list to hold parsed data
data_list = []

# Specify the file path
file_path = "C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/2022_env/TSG_Salinity_SBE38Temp/TSG-RAW_20220511-000000.Raw"

# Read and parse the file
with open(file_path, 'r') as file:
    for line in file:
        # Split the line into date, time, and the rest of the data
        date_time, *data_parts = line.strip().split(',')
        # Initialize a dictionary to hold parsed data for the current line
        line_data = {'DateTime': date_time}
        # Parse the rest of the data
        for part in data_parts:
            if '=' in part:
                key, value = part.split('=')
                line_data[key.strip()] = value.strip()
            else:
                # Handle the time part, which does not follow the key=value pattern
                # Assuming the first element in data_parts is always the time
                line_data['Time'] = part.strip()
        # Append the dictionary to the list
        data_list.append(line_data)

# Create a DataFrame
df = pd.DataFrame(data_list)

# Combine 'DateTime' and 'Time' into a single 'DateTime' column and convert to a datetime object
df['DateTime'] = pd.to_datetime(df['DateTime'] + ' ' + df.pop('Time'))

# Convert numeric columns to floats
numeric_cols = ['t1', 'c1', 's', 'sv', 't2']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

df.head()
output_csv_path = 'C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/2022_env/tsg11.2.csv'
df.to_csv(output_csv_path, index=False)

In [10]:
#Env 3 Fluorometer file,Chl-a units,  Source SBE45 SeabirdThermosalinograph  
import pandas as pd

# Define the path to your raw file
file_path = 'C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/Fluorometer/Chloro-MSG_20220511-000000.Raw'

# Initialize a list to hold the parsed data
data_list = []

# Open and read the raw file
with open(file_path, 'r') as file:
    for line in file:
        # Split the line by commas
        parts = line.strip().split(',')
        # Ensure the line has the expected number of parts
        if len(parts) >= 5:
            # Extract each part based on the known format
            date_str = parts[0]
            time_str = parts[1]
            sentence_label = parts[2]
            chlorophyll = parts[3]
            chl_dark_counts = parts[4]
            
            # Append a dictionary with the extracted values to the list
            data_list.append({
                'Date': date_str,
                'Time': time_str,
                'Sentence label': sentence_label,
                'chlorophyll (units: ug/l)': chlorophyll,
                'Chl dark counts': chl_dark_counts
            })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data_list)

# Combine 'Date' and 'Time' into a single 'DateTime' column if needed
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.drop(['Date', 'Time'], axis=1, inplace=True)

# Reorder the DataFrame to place 'DateTime' at the beginning if desired
df = df[['DateTime', 'Sentence label', 'chlorophyll (units: ug/l)', 'Chl dark counts']]

# Display the first few rows of the DataFrame
print(df.head())

# Optionally, save the DataFrame to a CSV file
output_csv_path = 'C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/Fluorometer/Chloro-MSG_20220511.csv'
df.to_csv(output_csv_path, index=False)


                 DateTime Sentence label chlorophyll (units: ug/l)  \
0 2022-05-11 00:00:01.178         $DERIV                      0.39   
1 2022-05-11 00:00:02.180         $DERIV                      0.39   
2 2022-05-11 00:00:03.181         $DERIV                      0.39   
3 2022-05-11 00:00:04.183         $DERIV                      0.39   
4 2022-05-11 00:00:05.184         $DERIV                      0.44   

  Chl dark counts  
0              32  
1              32  
2              32  
3              32  
4              36  


In [11]:
#Env file 4 Fluorometer file, Turbidity units, Source: SBE45 SeabirdThermosalinograh
import pandas as pd

# Define the path to your raw file
file_path = 'C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/Fluorometer/NTU-MSG_20220511-000000.Raw'

# Initialize an empty list to store the parsed data
data_list = []

# Open the raw file and read line by line
with open(file_path, 'r') as file:
    for line in file:
        # Remove any trailing newlines or whitespace and split the line by commas
        parts = line.strip().split(',')
        # Check if the line has the minimum expected number of parts
        if len(parts) >= 5:
            # Construct the datetime string by combining the date and time parts
            datetime_str = f"{parts[0]} {parts[1]}"
            # Extract the remaining pieces of data
            sentence_label = parts[2]
            ntu_count = parts[3]
            ntu_dark_count = parts[4]
            
            # Append the extracted data as a dictionary to the data list
            data_list.append({
                'DateTime': datetime_str,
                'sentence label': sentence_label,
                'NTU count (units: NTU)': ntu_count,
                'NTU dark count (cnts)': ntu_dark_count
            })

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(data_list)

# Convert the 'DateTime' column from string to datetime objects
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Optionally, convert NTU count and dark count to numeric types
df['NTU count (units: NTU)'] = pd.to_numeric(df['NTU count (units: NTU)'], errors='coerce')
df['NTU dark count (cnts)'] = pd.to_numeric(df['NTU dark count (cnts)'], errors='coerce')

# Display the first few rows of the DataFrame to verify
print(df.head())

# Optionally, save the DataFrame to a CSV file
output_csv_path = 'C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/Fluorometer/NTU-MSG_20220511.csv'
df.to_csv(output_csv_path, index=False)


                 DateTime sentence label  NTU count (units: NTU)  \
0 2022-05-11 00:00:01.188         $DERIV                    2.16   
1 2022-05-11 00:00:02.190         $DERIV                    2.16   
2 2022-05-11 00:00:03.191         $DERIV                    2.17   
3 2022-05-11 00:00:04.193         $DERIV                    2.16   
4 2022-05-11 00:00:05.194         $DERIV                    2.16   

   NTU dark count (cnts)  
0                    323  
1                    323  
2                    324  
3                    322  
4                    323  


In [28]:
#Read all the env files taht gonna make into metadata 
Lat11=pd.read_csv('C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/Lat_Long/processed_Lat_long_20220511.csv')
tsg11=pd.read_csv("C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/2022_env/tsg11.2.csv")
chlo11=pd.read_csv('C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/Fluorometer/Chloro-MSG_20220511.csv')
turb11=pd.read_csv('C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/Fluorometer/NTU-MSG_20220511.csv')
roi11=pd.read_csv("C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/2022_roi/D202205/D20220511 files.csv")                  

### First merge all the env files with high resolution, two dataframe at a time

In [29]:
#Lat11 and tsg11
#First change into DateTime format before merge
tsg11['DateTime']=pd.to_datetime(tsg11["DateTime"])
Lat11['DateTime']=pd.to_datetime(Lat11["DateTime"])

In [30]:
#Lat11 and tsg11
env1=pd.merge_asof(Lat11,tsg11,on="DateTime",tolerance=pd.Timedelta("1 seconds"))

In [32]:
env1.head()

,DateTime,Latitude,Longitude,t1,c1,s,sv,t2
0,2022-05-11 00:00:00,5421.8252N,16616.0461W,NaN,NaN,NaN,NaN,NaN
1,2022-05-11 00:00:01,5421.8286N,16616.0436W,5.1321,3.16051,32.7306,1466.430,4.6564
2,2022-05-11 00:00:02,5421.8318N,16616.0412W,5.1308,3.16077,32.7348,1466.447,4.6592
3,2022-05-11 00:00:03,5421.8351N,16616.0385W,5.1306,3.16067,32.7339,1466.457,4.6618
4,2022-05-11 00:00:04,5421.8383N,16616.0356W,5.1301,3.16072,32.7349,1466.461,4.6627


In [33]:
tsg11.head()

,DateTime,t1,c1,s,sv,t2
0,2022-05-11 00:00:00.688,5.1321,3.16051,32.7306,1466.430,4.6564
1,2022-05-11 00:00:01.729,5.1308,3.16077,32.7348,1466.447,4.6592
2,2022-05-11 00:00:02.770,5.1306,3.16067,32.7339,1466.457,4.6618
3,2022-05-11 00:00:03.812,5.1301,3.16072,32.7349,1466.461,4.6627
4,2022-05-11 00:00:04.852,5.1318,3.16080,32.7341,1466.465,4.6639


In [19]:
#chl11 and turb11

chlo11['DateTime']=pd.to_datetime(chlo11["DateTime"])
turb11['DateTime']=pd.to_datetime(turb11["DateTime"])

In [42]:
#check the merged dataframe
env1.head()

,DateTime,Latitude,Longitude,t1,c1,s,sv,t2
0,2022-05-11 00:00:00,5421.8252N,16616.0461W,NaN,NaN,NaN,NaN,NaN
1,2022-05-11 00:00:01,5421.8286N,16616.0436W,5.1321,3.16051,32.7306,1466.430,4.6564
2,2022-05-11 00:00:02,5421.8318N,16616.0412W,5.1308,3.16077,32.7348,1466.447,4.6592
3,2022-05-11 00:00:03,5421.8351N,16616.0385W,5.1306,3.16067,32.7339,1466.457,4.6618
4,2022-05-11 00:00:04,5421.8383N,16616.0356W,5.1301,3.16072,32.7349,1466.461,4.6627


In [20]:
#chlo11 and turb11
#First change into DateTime format before merge
env2=pd.merge_asof(chlo11,turb11,on="DateTime",tolerance=pd.Timedelta("1 seconds"))

In [45]:
#now merge of env1 and env2
env5_11=pd.merge_asof(env1,env2,on="DateTime",tolerance=pd.Timedelta("5 seconds"))

In [46]:
#checked the merged dataframe
env5_11.head()

,DateTime,Latitude,Longitude,t1,c1,s,sv,t2,Sentence label,chlorophyll (units: ug/l),Chl dark counts,sentence label,NTU count (units: NTU),NTU dark count (cnts)
0,2022-05-11 00:00:00,5421.8252N,16616.0461W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-05-11 00:00:01,5421.8286N,16616.0436W,5.1321,3.16051,32.7306,1466.430,4.6564,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-05-11 00:00:02,5421.8318N,16616.0412W,5.1308,3.16077,32.7348,1466.447,4.6592,$DERIV,0.39,32.0,NaN,NaN,NaN
3,2022-05-11 00:00:03,5421.8351N,16616.0385W,5.1306,3.16067,32.7339,1466.457,4.6618,$DERIV,0.39,32.0,$DERIV,2.16,323.0
4,2022-05-11 00:00:04,5421.8383N,16616.0356W,5.1301,3.16072,32.7349,1466.461,4.6627,$DERIV,0.39,32.0,$DERIV,2.16,323.0


In [43]:
#Now we can switch to lower resolution by merging with roifiles
#First convert it to same DateTime format and same name  to merge with env file
roi11["DateTime"]=pd.to_datetime(roi11["CombinedDateTime"])

In [47]:
#its time for merge
metadata5_11=pd.merge_asof(roi11,env5_11,on="DateTime",tolerance=pd.Timedelta("3 minutes"))

In [48]:
metadata5_11.head()

,Unnamed: 0,File Name,CombinedDateTime,DateTime,Latitude,Longitude,t1,c1,s,sv,t2,Sentence label,chlorophyll (units: ug/l),Chl dark counts,sentence label,NTU count (units: NTU),NTU dark count (cnts)
0,216,D20220511T000211_IFCB165.roi,2022-05-11 00:02:11,2022-05-11 00:02:11,5422.2528N,16615.6872W,5.1579,3.15960,32.6948,1466.756,4.7467,$DERIV,0.90,73.0,$DERIV,2.23,333.0
1,217,D20220511T002554_IFCB165.roi,2022-05-11 00:25:54,2022-05-11 00:25:54,5426.9211N,16611.7743W,5.4139,3.17712,32.6447,1467.482,4.9392,$DERIV,1.21,98.0,$DERIV,2.32,347.0
2,218,D20220511T004937_IFCB165.roi,2022-05-11 00:49:37,2022-05-11 00:49:37,5431.4913N,16607.9179W,5.3607,3.17497,32.6721,1467.143,4.8480,$DERIV,0.64,52.0,$DERIV,2.21,330.0
3,219,D20220511T011320_IFCB165.roi,2022-05-11 01:13:20,2022-05-11 01:13:20,5435.9941N,16603.9767W,5.5772,3.17979,32.5165,1467.779,5.0518,$DERIV,14.83,1206.0,$DERIV,2.97,444.0
4,220,D20220511T013704_IFCB165.roi,2022-05-11 01:37:04,2022-05-11 01:37:04,5440.4284N,16559.9609W,5.2565,3.13407,32.3069,1466.283,4.7522,$DERIV,13.43,1092.0,$DERIV,2.76,412.0


In [50]:
output_csv_path = 'C:/Users/kurta/OneDrive/Desktop/Spring_Mooring22_5may_1June/metadata22511_2.0.csv'
metadata5_11.to_csv(output_csv_path, index=False)